In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re
import psycopg2

In [2]:
earth_data = pd.read_csv('Mag6PlusEarthquakes_1900-2013.csv', parse_dates=True)

In [3]:
earth_data.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms
count,8313.000000,8313.000000,8313.000000,8312.000000,8111.000000,1098.000000,164.000000,1164.000000
mean,4.954110,47.350161,76.855975,6.564870,245.151399,30.418670,4.264072,1.007775
std,29.898265,122.446592,134.029768,0.482917,204.853588,19.698842,4.018516,0.186605
min,-71.000000,-179.990000,0.000000,6.000000,0.000000,8.700000,0.017966,0.000000
25%,-16.120000,-72.051000,15.000000,6.200000,63.000000,17.900000,1.476750,0.880000
50%,-0.534000,120.231000,32.100000,6.500000,221.000000,24.700000,3.271137,0.990000
75%,32.327000,146.430000,57.200000,6.900000,371.500000,36.900000,5.790000,1.130000
max,84.964000,180.000000,720.000000,9.600000,999.000000,230.400000,26.063000,1.680000


In [467]:
connection = psycopg2.connect(database='lab_4', user='alex', password='')
cursor = connection.cursor()

![title](lab_4.png)

In [468]:
cursor.execute("DROP TABLE IF EXISTS net")
cursor.execute("DROP TABLE IF EXISTS places")
cursor.execute("DROP TABLE IF EXISTS locations")
cursor.execute("DROP TABLE IF EXISTS magnitude")
cursor.execute("DROP TABLE IF EXISTS stations")
cursor.execute("DROP TABLE IF EXISTS earthquake")

In [469]:
event_list = list(map(lambda i: (earth_data['place'][i]), xrange(len(earth_data['longitude']))))
print len(event_list) 
print len(set(event_list))

8313
755


In [470]:
cursor.execute("CREATE TABLE net("+
               "net_id SERIAL UNIQUE,"+
               "net varchar(15) PRIMARY KEY)")

In [471]:
cursor.execute("CREATE TABLE places("
               "place_id SERIAL PRIMARY KEY,"+
               "place varchar(255))")

### Earthquake may happen in same points, of course not so frequently, but its not good for database table. Column 'place' have 755 unique values. May be improved by creating event table. Now primary key is 'local_id', bad choise but it is give posipility to quick import data to database for some tests.

In [472]:
cursor.execute("CREATE TABLE locations("+
               "local_id SERIAL UNIQUE,"+
               "latitude real NOT NULL,"+
               "longitude real NOT NULL,"+
               "depth real,"+
               "place int4 REFERENCES places(place_id),"+
               "net int4 REFERENCES net(net_id),"+
               "PRIMARY KEY(local_id))")

In [473]:
cursor.execute("CREATE TABLE magnitude("+
               "mag_id SERIAL UNIQUE,"+
               "magtype varchar(5) PRIMARY KEY)")

In [474]:
cursor.execute("CREATE TABLE stations("+
               "station_id SERIAL PRIMARY KEY,"+
               "nst int4,"+
               "gap real,"+
               "dmin real,"+
               "rms real)")

In [475]:
cursor.execute("CREATE TABLE earthquake("+
               "id varchar(255) PRIMARY KEY,"+
               "time timestamp NOT NULL,"+
               "location int4 REFERENCES locations(local_id),"+
               "mag float NOT NULL,"+
               "magnitude int4 REFERENCES magnitude(mag_id),"+
               "receive_station int4 REFERENCES stations(station_id),"+
               "update_time timestamp NOT NULL)"
                )

In [476]:
for i in earth_data:
    print i

time
Date
Time
latitude
longitude
depth
mag
magType
nst
gap
dmin
rms
net
id
updated
place
type


In [477]:
mag_list = list(set(earth_data['magType']))
print mag_list

[nan, 'Mwc', 'mwp', 'mww', 'mwr', 'Mwb', 'Mt', 'mb', 'mw', 'mj', 'mwc', 'mwb', 'Mw', 'Mwp', 'uk', 'ms', 'Mww']


In [478]:
# insert data into magnitude table
for mag in mag_list:
    cursor.execute("INSERT INTO magnitude (magtype) VALUES (%s)", (mag,))

In [479]:
net_list = list(set(earth_data['net']))
print net_list

['official', 'nc', 'us', 'centennial', 'atlas', 'pde']


In [480]:
# insert data into net table
for net in net_list:
    cursor.execute("INSERT INTO net (net) VALUES (%s)", (net,))

### For insert data with key, python have great dictionary realization. And it is way to do complex key in database.

In [481]:
dict_maker = lambda data_list: {key:item for key,item in zip(data_list, range(1,len(data_list)+1))}

In [482]:
place_list = list(set(earth_data['place']))

In [483]:
for place in place_list:
    cursor.execute("INSERT INTO places (place) VALUES (%s)", (place,))

In [484]:
net_dict = dict_maker(net_list)
mag_dict = dict_maker(mag_list)
place_dict = dict_maker(place_list)

In [485]:
for i in range(len(earth_data['longitude'])):
    cursor.execute("INSERT INTO locations (latitude, longitude, depth, place, net) VALUES (%s, %s, %s, %s, %s)",
                  (earth_data['latitude'][i], earth_data['longitude'][i], earth_data['depth'][i], 
                   place_dict[earth_data['place'][i]], net_dict[earth_data['net'][i]],))

In [486]:
earth_data['nst'] = earth_data['nst'].replace(np.nan, 0)
earth_data['gap'] = earth_data['gap'].replace(np.nan, 0)
earth_data['dmin'] = earth_data['dmin'].replace(np.nan, 0)
earth_data['rms'] = earth_data['rms'].replace(np.nan, 0)

In [487]:
for i in range(len(earth_data['longitude'])):
        cursor.execute("INSERT INTO stations (nst, gap, dmin, rms) VALUES (%s, %s, %s, %s)", 
                       (earth_data['nst'][i],earth_data['gap'][i],earth_data['dmin'][i],earth_data['rms'][i]),)

In [491]:
for i in range(len(earth_data['longitude'])):
    cursor.execute("INSERT INTO earthquake (id, time, location, mag, magnitude, receive_station, update_time) VALUES (%s, %s, %s, %s, %s, %s, %s)", 
                       (earth_data['id'][i],earth_data['time'][i],place_dict[earth_data['place'][i]], 
                        earth_data['mag'][i], mag_dict[earth_data['magType'][i]], i+1, earth_data['updated'][i]))

In [489]:
connection.commit()
cursor.close()
connection.close()